In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'

In [ ]:
import getpass
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral", temperature=0)

In [ ]:
llm.invoke("What is LangGraph?")


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_path="https://blog.langchain.dev/langgraph/"
)
docs = loader.load()

In [ ]:
docs[0].page_content

In [ ]:
len(docs[0].page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)

In [ ]:
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
len(all_splits[1].page_content)

In [ ]:
from langchain_community import embeddings

embedding = embeddings.OllamaEmbeddings(
    model="nomic-embed-text"
)

In [ ]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding,
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [ ]:
retriever.get_relevant_documents(
    "What is LangGraph?"
)

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is LangGraph?")


In [ ]:
rag_chain.invoke("What is LangGraph used for?")